# Packages

In [ ]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from scipy.special import logsumexp, gammaln
from sklearn.preprocessing import MultiLabelBinarizer

try:
    import google.colab
    IN_COLAB = True
    path = '/content/drive/MyDrive/PhD/Modules/CS5340 Uncertainty Modeling in AI/Project/'
except:    
    IN_COLAB = False
    path = './'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

In [ ]:
users_ds = np.load(path + 'users_ds_dense.npy', allow_pickle=True)

In [ ]:
# size of training dataset
U, T, N = users_ds.shape
print('Shape of dataset:', users_ds.shape)
print('Number of users:', U)
print('Number of movie titles:', N)
print('Number of periods:', T)

Shape of dataset: (1212, 73, 17768)
Number of users: 1212
Number of movie titles: 17768
Number of periods: 73


In [ ]:
t_predict = -1                                          # index for holdout period for prediction
test_ds = users_ds[:,t_predict,:]                       # extract holdout period from dataset
test_ds = test_ds[:,np.newaxis,:]                       # align # of dimensions
users_ds = users_ds[:,:t_predict,:]                     # specify training periods

In [ ]:
# calculate and store total number of ratings per user in a period
users_Nt = np.sum(users_ds, axis=-1)    # number of movies rated by user in each time period
T = np.shape(users_ds)[1]               # final length of time period after trimming
users_Nt.shape 

(1212, 72)

# Helper Functions

In [ ]:
rng = np.random.default_rng()

In [ ]:
def logdotexp(A, B):
    max_A = np.max(A)
    max_B = np.max(B)
    C = np.matmul(np.exp(A - max_A), np.exp(B - max_B))
    np.log(C, out=C)
    C += max_A + max_B
    return C

In [ ]:
def initialise_params_and_probs(pi_alpha, A_alpha, theta_alpha):
    K = len(pi_alpha)
    pi = rng.dirichlet(alpha=pi_alpha, size=1).squeeze()                        # Initial state distribution
    A = np.zeros((K,K))                                                         # Transitional probabilities
    theta = np.zeros((K,N))                                                     # Multinomial probabilities
    for k in range(K):
        A[k,:] = rng.dirichlet(alpha=A_alpha[k,:], size=1).squeeze()
        theta[k,:] = rng.dirichlet(alpha=theta_alpha[k,:], size=1).squeeze()
    pi = np.log(pi)
    A = np.log(A)
    theta = np.log(theta)

    # NBD parameters
    a = np.ones((K))
    p = rng.uniform(low=0.7, high=0.9, size=(K))  # probability for event tied to N
    b = p / (1-p)
    return pi, A, theta, a, b

In [ ]:
def initialise_latent_states(pi, A, U, T):
    # latent states of all users across the periods
    latent_states = np.empty((U,T), dtype=int)
    latent_states[:,0] = rng.multinomial(n=1, pvals=np.exp(pi), size=(U,)).argmax(axis=1)    # initial latent state assignments
    sample_A = np.tile(np.exp(A), (U,1,1))
    for t in range(T-1):
        previous_states = latent_states[:,t]
        pvals = sample_A[t, previous_states] 
        latent_states[:,t+1] = rng.multinomial(n=1, pvals=pvals).argmax(axis=1) 
        
    return latent_states

# latent_states = initialise_latent_states(pi, A, U, T)
# print(latent_states.shape, latent_states[0])

In [ ]:
def update_b(latent_states, users_Nt, K, a):
    alpha = np.ones((K), dtype=int)
    beta = users_Nt.shape[0] * users_Nt.shape[1] + 1

    b = np.zeros((K))
    for i in range(K):
        indices = latent_states == i
        alpha[i] += users_Nt[indices].sum()
        p = rng.beta(alpha[i], beta, 1)
        b[i] = p / (1-p)

    return b

# b = update_b(latent_states, users_Nt, K, 1)
# print(b)

In [ ]:
def update_theta(latent_states, dataset, K):
    alpha = np.ones((K,N), dtype=int)
    U = latent_states.shape[0]
    for u in range(U):
        user_states = latent_states[u,:]
        counts = dataset[u,:,:]
        for t in range(T):
            alpha[user_states[t]] += counts[t]

    # print(alpha.sum() - K*N, dataset.sum()) # checksum

    # theta = rng.dirichlet(alpha=alpha, size=1) # dirichlet does not accept array like for alpha unlike multinomial

    theta = np.zeros((K,N))
    for k in range(K):
        theta[k,:] = rng.dirichlet(alpha=alpha[k,:], size=1).squeeze()

    return np.log(theta)

# theta = update_theta(latent_states, users_ds, K)
# np.exp(theta).sum(axis=-1)

In [ ]:
def update_A(latent_states, K):
    alpha = np.ones((K,K), dtype=int)
    T = latent_states.shape[1]

    for t in range(T - 1):
        current_states = latent_states[:,t]
        next_states = latent_states[:,t+1]
        for i in range(len(current_states)):
            alpha[current_states[i], next_states[i]] += 1
    
    # print(alpha.sum() - K*K, U*(T-1)) # checksum

    A = np.zeros((K,K))
    for k in range(K):
        A[k,:] = rng.dirichlet(alpha=A_alpha[k,:], size=1).squeeze()
    return np.log(A)

# A = update_A(latent_states, K)
# np.exp(A)

In [ ]:
def update_pi(latent_states, K):
    alpha = np.ones((K), dtype=int)

    for i in range(K):
        alpha[i] += (latent_states[:,0] == i).sum()

    # print(alpha.sum() - K, U)

    pi = rng.dirichlet(alpha=alpha, size=1).squeeze()

    return np.log(pi)

# pi = update_pi(latent_states, K)
# print(np.exp(pi))

In [ ]:
def calculate_emission_prob(users_ds, users_Nt, nbd_first_part, multi_first_part, theta, a, b):
    
    # log prob of N given z as gamma mixture of poisson i.e. number of articles read
#     p_n_ab = gammaln(users_Nt[..., np.newaxis] + a[np.newaxis, np.newaxis, ...]) \
#             - gammaln(a)[np.newaxis, np.newaxis, ...] - gammaln(users_Nt+1)[..., np.newaxis] \
#             + users_Nt[..., np.newaxis] * np.log(b)[np.newaxis, np.newaxis, ...]  \
#             - (users_Nt[..., np.newaxis] + a[np.newaxis, np.newaxis, ...]) * np.log(b+1)[np.newaxis, np.newaxis, ...]

    second_part = users_Nt[..., np.newaxis] * np.log(b)[np.newaxis, np.newaxis, ...]  \
                - (users_Nt[..., np.newaxis] + a[np.newaxis, np.newaxis, ...]) * np.log(b+1)[np.newaxis, np.newaxis, ...]
    
    p_n_ab = nbd_first_part + second_part

    # log prob of I given z and N as Multinomial(theta) i.e. which movies are rated=1/unrated=0
    second_part = np.matmul(users_ds, theta.T)
    p_i_theta = multi_first_part + second_part
    del second_part

    # log prob of joint dist of N, I given z
    p_i_z = p_n_ab + p_i_theta

    return p_i_z, p_n_ab, p_i_theta

In [ ]:
def calculate_posterior(A, pi, p_i_z, U, T, K):

    # Calculate normalised alpha and beta
    alpha = np.empty((U,T,K), dtype='float64')
    p_i_i = np.empty((U,T), dtype='float64')

    alpha[:,0] = p_i_z[:,0] + pi
    alpha[:,0] -= logsumexp(alpha[:,0], axis=-1)[...,np.newaxis]
    for t in range(1, T):
        alpha[:,t] = logdotexp(alpha[:,t-1], A) + p_i_z[:,t]
        p_i_i[:,t] = logsumexp(alpha[:,t], axis=-1)
        alpha[:,t] -= p_i_i[:,t][...,np.newaxis]

    beta = np.zeros((U,T,K), dtype='float64')
    for u in range(U):
        for t in range(T-2, -1, -1):
            beta[u,t] = logdotexp(A, (p_i_z[u,t+1] + beta[u,t+1]))
            beta[u,t] -= p_i_i[u,t+1] # normalization

    # numerical issues "divide by zero encountered in log" for the vectorized code below
    # for t in range(T-2, -1, -1):
    #     beta[:,t,:] = logdotexp((p_i_z[:,t+1,:] + beta[:,t+1,:]), A.T)
    #     beta[:,t,:] -= p_i_i[:,t+1][...,np.newaxis] # normalization

    # log prob of Z(t) given I(1:T)
    p_z_i = alpha + beta

    # log prob of Z(t), Z(t+1) given I(1:T)
    p_zz_i = np.empty((U,T-1,K,K), dtype='float64')
    for u in range(U):
        for t in range(T-1):
            p_zz_i[u,t,:,:] = np.tile(alpha[u,t,:], (K,1)).T + A + np.tile(p_i_z[u,t+1,:], (K,1)) + np.tile(beta[u,t+1,:], (K,1)) 
            p_zz_i[u,t,:,:] -= p_i_i[u,t+1][...,np.newaxis,np.newaxis] # normalization
    
    return p_z_i, p_zz_i

In [ ]:
def update_all_states(latent_states, p_z_i, p_zz_i):
    U, T = latent_states.shape

    sample_pi = np.exp(p_z_i[:,0,:]).mean(axis=0)    
    latent_states[:,0] = rng.multinomial(n=1, pvals=sample_pi, size=(U,)).argmax(axis=1)

    sample_A = np.exp(p_zz_i - p_z_i[:,:-1,:][...,np.newaxis]).mean(axis=(0))
    for t in range(T-1):
        previous_states = latent_states[:,t]
        pvals = sample_A[t, previous_states]
        latent_states[:,t+1] = rng.multinomial(n=1, pvals=pvals).argmax(axis=1)

    # unnormalised_pi = pi[np.newaxis,...] + p_i_z[:,0,:] + beta[:,0,:]
    # unnormalised_pi = np.exp(unnormalised_pi).mean(axis=0)
    # normalised_pi = unnormalised_pi / unnormalised_pi.sum()    

    # for t in range(T-1):
        #     unnormalised_A = A[np.newaxis,...] + np.expand_dims(p_i_z[:,t,:], 1) + np.expand_dims(beta[:,t,:], 1)
        #     unnormalised_A = np.exp(unnormalised_A).mean(axis=0)
        #     normalised_A = unnormalised_A / unnormalised_A.sum(axis=1)[np.newaxis,...]
        #     if t<2: print(normalised_A)

    return latent_states

In [ ]:
def calculate_log_likelihood(pi, A, p_n_ab, p_i_theta, p_z_i, p_zz_i):
    # intial state 
    init = np.sum(np.multiply(np.exp(p_z_i[:,0]), pi[np.newaxis,...]))

    # transitional 
    trans = np.sum(np.multiply(np.exp(p_zz_i), A[np.newaxis, np.newaxis,...]))

    # # of items 
    nbd = np.sum(np.multiply(np.exp(p_z_i), p_n_ab))

    # specific item 
    multi = np.sum(np.multiply(np.exp(p_z_i), p_i_theta))
    
    return init + trans + nbd + multi

# Gibbs Sampling

In [ ]:
def gibbs_sampling(users_ds, users_Nt, pi_alpha, A_alpha, theta_alpha, n_iterations, burn_in=50):
    max_likelihood = None
    U, T = users_ds.shape[:-1]
    K = len(pi_alpha)

    pi, A, theta, a, b = initialise_params_and_probs(pi_alpha, A_alpha, theta_alpha)
    latent_states = initialise_latent_states(pi, A, U, T)

    # store repeated calculations in NBD and multinomial log prob for significant speed up
    multi_first_part = (gammaln(users_Nt + 1) - gammaln(users_ds + 1).sum(axis=-1))[..., np.newaxis]
    nbd_first_part = gammaln(users_Nt[..., np.newaxis] + a[np.newaxis, np.newaxis, ...]) \
            - gammaln(a)[np.newaxis, np.newaxis, ...] - gammaln(users_Nt+1)[..., np.newaxis] 

    pi_bar = np.zeros(pi.shape)
    A_bar = np.zeros(A.shape)
    theta_bar = np.zeros(theta.shape)
    b_bar = np.zeros(b.shape)

    # GIBBS SAMPLING
    start_time = datetime.now() # for keeping track of running time
    for iteration in range(n_iterations + burn_in):

        # UPDATE PARAMETERS AND PROBABILITIES
        b = update_b(latent_states, users_Nt, K, a[0])
        theta = update_theta(latent_states, users_ds, K)
        A = update_A(latent_states, K)
        pi = update_pi(latent_states, K)

        # UPDATE LATENT STATES    
        p_i_z, p_n_ab, p_i_theta = calculate_emission_prob(users_ds, users_Nt, nbd_first_part, multi_first_part, theta, a, b)
        p_z_i, p_zz_i = calculate_posterior(A, pi, p_i_z, U, T, K)
        latent_states = update_all_states(latent_states, p_z_i, p_zz_i)

        # CALCULATE EXPECTED LOG LIKELIHOOD
        likelihood = calculate_log_likelihood(pi, A, p_n_ab, p_i_theta, p_z_i, p_zz_i)
        if max_likelihood is None: 
            max_likelihood = likelihood
        else:
            if likelihood > max_likelihood: 
                max_likelihood = likelihood
                pi_max = pi
                A_max = A
                theta_max = theta
                a_max = a
                b_max = b

        if np.isnan(likelihood):
            print('Numerical issues in calculation of log likelihood')
            break
        print('Iteration', iteration+1,': log likelihood =', likelihood)

        if iteration+1 > burn_in:
            pi_bar += pi
            A_bar += A
            theta_bar += theta
            b_bar += b

    pi_bar /= n_iterations
    A_bar /= n_iterations
    theta_bar /= n_iterations
    b_bar /= n_iterations

    run_time = datetime.now() - start_time    
    print('Execution time for Gibbs Sampling iterations:', run_time)

    return (pi_bar, A_bar, theta_bar, a, b_bar), (pi_max, A_max, theta_max, a_max, b_max)

In [ ]:
n_iterations = 500
K = 15

prior_const = 0.9*K                                       # affects the parameters of the Dirichlet priors
pi_alpha = prior_const/K * np.ones((K))                 # alpha hyperparams for pi
A_alpha = prior_const/K * np.ones((K,K))                # alpha hyperparams for A
theta_alpha = prior_const/K * np.ones((K,N))            # alpha hyperparams for theta

expected_params, max_params = gibbs_sampling(users_ds, users_Nt, pi_alpha, A_alpha, theta_alpha, n_iterations, burn_in=100)
pi_bar, A_bar, theta_bar, a_bar, b_bar = expected_params
pi_max, A_max, theta_max, a_max, b_max = max_params

ValueError: ignored

In [ ]:
# save parameters

np.save(path + 'pi_bar_K_' + str(K), pi_bar)
np.save(path + 'A_bar_K_' + str(K), A_bar)
np.save(path + 'theta_bar_K_' + str(K), theta_bar)
np.save(path + 'a_bar_K_' + str(K), a_bar)
np.save(path + 'b_bar_K_' + str(K), b_bar)

np.save(path + 'pi_max_K_' + str(K), pi_max)
np.save(path + 'A_max_K_' + str(K), A_max)
np.save(path + 'theta_max_K_' + str(K), theta_max)
np.save(path + 'a_max_K_' + str(K), a_max)
np.save(path + 'b_max_K_' + str(K), b_max)

In [ ]:
# Use either the expected parameter values or the max likelihood parameteres

pi, A, theta, a, b = pi_bar, A_bar, theta_bar, a_bar, b_bar

# pi, A, theta, a, b = pi_max, A_max, theta_max, a_max, b_max

# Evaluation on test data

In [ ]:
multi_first_part = (gammaln(users_Nt + 1) - gammaln(users_ds + 1).sum(axis=-1))[..., np.newaxis]
nbd_first_part = gammaln(users_Nt[..., np.newaxis] + a[np.newaxis, np.newaxis, ...]) \
        - gammaln(a)[np.newaxis, np.newaxis, ...] - gammaln(users_Nt+1)[..., np.newaxis] 
p_i_z, p_n_ab, p_i_theta = calculate_emission_prob(users_ds, users_Nt, nbd_first_part, multi_first_part, theta, a, b)
p_z_i, p_zz_i = calculate_posterior(A, pi, p_i_z, U, T, K)
likelihood = calculate_log_likelihood(pi, A, p_n_ab, p_i_theta, p_z_i, p_zz_i)
print(likelihood)

In [ ]:
# number of items to recommend
num_items = 5000

# log prob of user each latent class in next period assuming user in Z(t) with log p(Z(t)|I(1:T))
# result is multiplying transitional prob to prob of user in each latent class at time t
p_z = logdotexp(p_z_i[:,-1], A)

# calculate probability that item i is not read in the next time period
p_noti_z = np.power(1 + b[...,np.newaxis] * np.exp(theta), -a[...,np.newaxis])

# calculate rank score of the items likely to appear in next time period
rank_score = -np.exp(p_z) @ p_noti_z

# generate indices of top num_items to recommend which will be unsorted
rec_list = np.argpartition(rank_score, -num_items, axis=-1)[:,-num_items:]

# sort indices by rank score
rec_list_score = np.array([row[rec_list[i,:]] for i, row in enumerate(rank_score)]) # get the scores of items in rec_list
sorted_rec_list = np.array([row[np.flip(np.argsort(rec_list_score[i]))] for i, row in enumerate(rec_list)]) # sort the rec_list based on the score

# check if item in user history
user_history = np.array([row[:,sorted_rec_list[i]] for i, row in enumerate(users_ds)]) # get all values in user_ds corresponding to the item in rec_list for each user in each time period
user_history = np.sum(user_history, axis=1) # get boolean array indicating whether each item in sorted_rec_list is in user history (assumes user only has each item at most once)
if user_history.max() > 1: print('There are repeated ratings of a movie by at least one user')
# print(user_history.shape)

# filter sorted_rec_list for items not in user history
filtered_rec_list = [row[np.logical_not(user_history[i])] for i, row in enumerate(sorted_rec_list)] # each user's list will not have the same amount of items as it depends on user history

# get multi-hot encoding of top N recommended movies for the next period
mlb = MultiLabelBinarizer(classes=range(N), sparse_output=False) # prediction done on based on one hot encoding indexing i.e. starting index is 0
top_5_list = [mlb.fit_transform([user[:5]]) for user in filtered_rec_list] # convert top 5 list to one hot encoding
top_10_list = [mlb.fit_transform([user[:10]]) for user in filtered_rec_list] 

# test how many of top N recommended movies appear in user's rated list of movies in the test period
positive_top_5 = [np.multiply(test_ds[i], rec_user) for i, rec_user in enumerate(top_5_list)] # get (#users,#items) boolean vectors indicating whether recommended movie was rating in test period
users_result_top_5 = [row.sum() for row in positive_top_5] # get list of positive matches per user
all_result_top_5 = np.sum(users_result_top_5) # total number of positive matches across all users

positive_top_10 = [np.multiply(test_ds[i], rec_user) for i, rec_user in enumerate(top_10_list)] 
users_result_top_10 = [row.sum() for row in positive_top_10] # get list of positive matches per user
all_result_top_10 = np.sum(users_result_top_10) # total number of positive matches across all users

# total number of movies rated in test period
test_num_movies_rated = np.sum(test_ds).sum()

# output results to excel via pandas df
dict_result = {'Log Likelihood':likelihood, 'Iterations':n_iterations,            
            '# movies rated in test period': test_num_movies_rated, 
            'Total +ve for top 5':all_result_top_5, 
            'Precision of top 5':all_result_top_5/(5*U),
            'Recall of top 5':all_result_top_5/test_num_movies_rated,
            'Total +ve for top 10':all_result_top_10,
            'Precision of top 10':all_result_top_10/(10*U),
            'Recall of top 10':all_result_top_10/test_num_movies_rated
            }
df_result = pd.DataFrame(data=dict_result, index=[0])
print(df_result)
df_result.to_csv(path + 'result.csv', index=False)